# VK API
## получение токена

Опишу здесь алгоритм получения токена, для себя, на всякий случай:

1) переходим по ссылке -- https://vk.com/dev/first_guide?f=3.%20%D0%90%D0%B2%D1%82%D0%BE%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F%20%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8F

2) находим "2. Регистрация приложения", там - ссылка на создание приложения -- https://vk.com/editapp?act=create

3) переходим, называем приложуху как угодно, выбираем пункт "Stand-alone", создаём приложение (я назвала своё Roz_GP)

4) на следующей появившейся странице "Информация" сохраняем изменения

5) Заходим в "Настройки" (боковая панель слева, вверху - название нашего приложения). В настройках -- сервисный ключ доступа. Его и используем как TOKEN в дальнейшем. Он действует сутки, потом нужно будет вернуться и там же, в "Настройках", нажать на круговые стредочки рядом с ним и таким образом сгенерировать новый, сохранив изменения!

In [19]:
import requests

In [20]:
from time import sleep # решает 'error_msg': 'Too many requests per second'

In [21]:
# Загрузка нужных библиотек
import pandas as pd
import re

In [22]:
!pip install openpyxl

In [23]:
TOKEN = '' # впиши свой!

*Примечание: откройте любую фотографию пользователя/сообщества; первые цифры после слова photo (XXXXXX в ссылке вида photoXXXXXX_YYYYYYY) — это и есть интересующий вас ID.*

"Как тебя зовут" - https://vk.com/tebya_zovyt
id - 72296316

"Убойные истории" - https://vk.com/killer_stories
id - 55544604


"Подслушано Метро Москвы" - https://vk.com/pmmgo
id - 144260965

In [24]:
domain = 'tebya_zovyt'
Q = 1500
owner = -72296316

In [25]:
def request_post(domain, offset):
    if offset != 0:
        data_posts = requests.get(
            'https://api.vk.com/method/wall.get', 
            params={
                'domain': domain,
                'offset': offset,
                'count': 100,
                'v':'5.154',
                'access_token': TOKEN

            }
        ).json()
    else:
        data_posts = requests.get(
            'https://api.vk.com/method/wall.get', 
            params={
                'domain': domain,
                'count': 100,
                'v':'5.154',
                'access_token': TOKEN

            }
        ).json()
    return data_posts

In [26]:
%%time

data_posts = []
for i in range(0, Q, 100):
    data_piece = request_post(domain, i)
    data_posts.extend(data_piece['response']['items'])
    sleep(0.5)

Wall time: 21.6 s


In [27]:
len(data_posts)

1500

In [28]:
# Достанем id каждого поста (чтобы потом достать комменты)
# ещё сопоставим текст поста и его id в словаре
def get_post_ids(data):
    posts_id2text = {}
    for x in range(len(data_posts)):
        post_id = data[x]['id']
        post_text = data[x]['text']
        posts_id2text[post_id] = post_text
    return posts_id2text

In [29]:
posts_id2text = get_post_ids(data_posts)

In [30]:
len(posts_id2text)

1500

### Достаём комменты из-под каждого поста:

Oтделяем комменты первого поста от всех остальных, чтобы получить необходимую форму записи инфы, а потом достаём комменты из всех остальных постов и в виде элементов добавляем их в список [responce][items] словаря data_comments.

*Примечание: у метода wall.getComments тоже есть параметр offset!*

In [31]:
def get_comments(owner, posts_dict):
    data_comments = []
    for ind, pid in enumerate(posts_dict.keys()):
        if ind == 0:
            first_comments = requests.get(
            'https://api.vk.com/method/wall.getComments', 
            params={
                'owner_id': owner,
                'post_id': pid,
                'need_likes': True,
                'count': 100,
                'v':'5.154',
                'access_token': TOKEN
                }
            ).json()
            try:    
                data_comments.extend(first_comments['response']['items'])
            except Exception:
                continue

        else:   
            comment_info = requests.get(
                'https://api.vk.com/method/wall.getComments', 
                params={
                    'owner_id': owner,
                    'post_id': pid,
                    'need_likes': True,
                    'count': 100,
                    'v':'5.154',
                    'access_token': TOKEN
                }
            ).json()
            sleep(0.5)
            try:    
                data_comments.extend(comment_info['response']['items'])
            except Exception:
                continue
    return data_comments

In [32]:
%%time

data_comments = get_comments(owner, posts_id2text)

Wall time: 18min 32s


In [33]:
len(data_comments)

66632

In [34]:
# пример: как выглядят данные
data_comments[712]

{'id': 2404720,
 'from_id': 102037994,
 'date': 1699467286,
 'text': 'Красивые детки',
 'post_id': 2404681,
 'owner_id': -72296316,
 'parents_stack': [],
 'likes': {'can_like': 1, 'count': 3, 'user_likes': 0},
 'thread': {'count': 0,
  'items': [],
  'can_post': True,
  'show_reply_button': True}}

In [35]:
import json

In [36]:
with open('data_comments.json', 'w') as f:
    json.dump(data_comments, f)

In [37]:
# Opening JSON file
f = open('data_comments.json')

# returns JSON object as a dictionary
data_comments = json.load(f)

# Closing file
f.close()

#### Чтобы получать ветки комментариев, нужно делать отдельные запросы. 

#### Внимание! В ответ на эти запросы мы получим только ответы на комментарии из переменной data_comments, а сами комменты-родители туда не попадут.

In [38]:
# словарь "id коммента + id поста"
com_post_dict = {}
for item in data_comments:
    try:
        com_id = item['id']
        post_id = item['post_id']
        com_post_dict[com_id] = post_id
    except Exception:
        continue

In [39]:
len(com_post_dict.items())

66091

In [40]:
def get_threads(owner, dict_cp):
    data_threads = []
    for n, item in enumerate(list(dict_cp.items())):
        if n == 0:
            first_thread = requests.get(
            'https://api.vk.com/method/wall.getComments', 
            params={
                'owner_id': owner,
                'post_id': item[1],
                'need_likes': True,
                'count': 100,
                'comment_id': item[0],
                'v':'5.154',
                'access_token': TOKEN
                }
            ).json()
            try:    
                data_threads.extend(first_thread['response']['items'])
            except Exception:
                continue
        else:
            threads_info = requests.get(
            'https://api.vk.com/method/wall.getComments', 
            params={
                'owner_id': owner,
                'post_id': item[1],
                'need_likes': True,
                'count': 100,
                'comment_id': item[0],
                'v':'5.154',
                'access_token': TOKEN
                }
            ).json()
            sleep(0.2)
            try:    
                data_threads.extend(threads_info['response']['items'])
            except Exception:
                continue
    return data_threads

In [41]:
%%time

data_threads = get_threads(owner, com_post_dict)

Wall time: 5h 46min 20s


In [42]:
len(data_threads)

78989

In [43]:
data_threads[1109]

{'id': 2404124,
 'from_id': 11457660,
 'date': 1699456102,
 'text': '[id46422029|Евгений], конгрессы обычно - доклады специалистов для специалистов. А я иронично заметила: всем рядовым гражданам мира... Впрочем, спасибо! Специально озадачилась этим вопросом. Прошел совсем вот на днях, правда с формулировкой "Всемирный русскоязычных..."',
 'post_id': 2402386,
 'owner_id': -72296316,
 'parents_stack': [2402418],
 'likes': {'can_like': 1, 'count': 0, 'user_likes': 0},
 'reply_to_user': 46422029,
 'reply_to_comment': 2403570}

In [44]:
with open('data_threads.json', 'w') as f:
    json.dump(data_threads, f)

In [45]:
# Opening JSON file
f = open('data_threads.json')

# returns JSON object as a dictionary
data_threads = json.load(f)

# Closing file
f.close()

#### Теперь попробуем совместить комментарии и треды к ним. 

Хочу создать таблицу, где коментарии вместе с их тредами будут как отдельные диалоги, со столбцами: 

    А. номер диалога (дублируется на каждой строчке треда)
    Б. id поста (пусть будет; дублируется на каждой строчке треда)
    В. id комментария
    Г. id комментатора
    Д. текст комментария
    Е. id сообщества (пусть будет)

In [46]:
# соберём словарь соответствия 
# комментов-родителей из data_comments
# и тредов из data_threads

parent_to_threads = {}

for item in data_comments:
    parent_id = item['id']
    threads = [thread_com['id'] for thread_com in data_threads 
               if parent_id in thread_com['parents_stack']]
    parent_to_threads[parent_id] = threads

In [47]:
len(parent_to_threads)

66632

In [48]:
def create_df_line(X, num):
    try:
        post_id = X['post_id']
        post_text = posts_id2text[post_id]
        comment_id = X['id']
        author_id = X['from_id']
        text = X['text']
        source_id = X['owner_id']
        line = [num, post_id, post_text, comment_id, author_id, text, source_id]
    except KeyError:
        line = [num,0,0,0,0,0,0]
    return line

In [49]:
def get_info_by_id(id2go, dataname):
    result = [item for item in dataname if item['id'] == id2go][0]
    return result

In [50]:
%%time

df_data = []
N = 0

parent_comments = data_comments

for unit in list(parent_to_threads.items()):
    parent = get_info_by_id(unit[0], data_comments)
    
    df_data.append(create_df_line(parent, N))
    
    if len(unit[1]) != 0:
        for answer in unit[1]:
            tread_comment = get_info_by_id(answer, data_threads)
            df_data.append(create_df_line(tread_comment, N))
    
    N +=1

Wall time: 21min 26s


In [51]:
df = pd.DataFrame(df_data, columns=['N_dialogue','post_id','post_text','comment_id','author_id','text','source_id'])

In [52]:
df

,N_dialogue,post_id,post_text,comment_id,author_id,text,source_id
0,0,2404074,"мои потешные фотокарточки родом из детства, ко...",2404078,198923241,Шикарные фотографии! Вызывают улыбку!) 🥹🥰 Каже...,-72296316
1,1,2404074,"мои потешные фотокарточки родом из детства, ко...",2404079,322300243,👍🤣🤣🤣повеселили!,-72296316
2,2,2404074,"мои потешные фотокарточки родом из детства, ко...",2404080,322476095,😍😍😍,-72296316
3,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404082,137529301,Сколько радости на лице у жениха🙄,-72296316
4,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404093,137529301,"[id137529301|Sveta], костюмеру 5+🤌 отдельно за...",-72296316
5,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404094,269330820,"[id137529301|Sveta], мне кажется, невеста стар...",-72296316
6,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404107,11730534,"[id137529301|Sveta], готовый мем😂",-72296316
7,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404118,729973106,"[id137529301|Sveta], 🤔Чисто мужское отношение ...",-72296316
8,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404120,61023827,"[id137529301|Sveta], в эфире модный приговор 🔥🤩😂",-72296316
9,3,2404074,"мои потешные фотокарточки родом из детства, ко...",2404143,9206257,"[id137529301|Sveta], разрешили невесту поцелов...",-72296316


In [53]:
df.to_excel(f'VK_{domain}_vol2.xlsx')

C:\Users\sara7\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:915: UserWarning: Ignoring URL 'https://vk.com/id19562500,%20https://vk.com/id62916658-%20в%20начале%20комментариев%20мошенницы,не%20ведитесь%20на%20жалобные%20посты%20и%20фото%20собачек,эти%20фото%20уже%20на%20многих%20страничках%20постят,работают%20в%20паре,а%20то%20ещё%20кто%20нибудь%20додумается%20денег%20кинуть)' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
